The libary

In [ ]:
import arcpy
from arcpy import env  
from arcpy.sa import *


Setting up the workspace

In [ ]:
workspace = r"C:/Users/jurge/Desktop/Stellenbosch/2023(Hons)/Modules/GIT 716/Hackathon/Dataoutputs/WeatherData"
arcpy.env.workspace = workspace

#workspace = where some of the outputs are saved my workspace was where the csv data was stored, other files where in the workspace

Setting up enviorments

In [ ]:
input_gdb = "C:/Users/jurge/Desktop/Stellenbosch/2023(Hons)/Modules/GIT 716/Hackathon/Dataoutputs/WeatherData/Data Wrangled Meteocsv.csv"
output_feature_class = "weather_points.shp"
x_field = "longitude"
y_field = "latitude"
arcpy.management.XYTableToPoint(input_gdb, output_feature_class, x_field, y_field)

env.workspace = "C:/Users/jurge/Desktop/Stellenbosch/2023(Hons)/Modules/GIT 716/Hackathon/Dataoutputs/WeatherDATAshapefile"
arcpy.env.extent = "C:/Users/jurge/Desktop/Stellenbosch/2023(Hons)/Modules/GIT 716/Hackathon/Dataoutputs/250m_Extent_WGS84.tif"
arcpy.env.snapRaster = "C:/Users/jurge/Desktop/Stellenbosch/2023(Hons)/Modules/GIT 716/Hackathon/Dataoutputs/250m_Extent_WGS84.tif"
cellsize = "C:/Users/jurge/Desktop/Stellenbosch/2023(Hons)/Modules/GIT 716/Hackathon/Dataoutputs/250m_Extent_WGS84.tif"

#env.workspace = this will save the created points in the shapefile forder, the shapefile folder will create points from the csv file for each of the days
#arcpy.env.extent = will use the extend of the raster jess made
#arcpy.env.snapRaster = will snap to the raster jess made
#cellsize = will set the cellsize the same as the raster Jess made

Creating some input variables

In [ ]:
# Convert CSV to shapefile using XYTableToPoint
input_features = "weather_points.shp"
x_field = "longitude"
y_field = "latitude"
savelocation = "C:/Users/jurge/Desktop/Stellenbosch/2023(Hons)/Modules/GIT 716/Hackathon/Dataoutputs/InterpolateOut/"

#savelocation = where the interpolation rasters must be saved

List of date

In [ ]:
cursor_fields = ["date"]  # Add other fields if needed
dates = set()  # Use a set to store unique dates will then only make each date show up once

with arcpy.da.SearchCursor(input_features, cursor_fields) as cursor:
    for row in cursor:
        date_value = row[0]  
        dates.add(date_value)
        
#this code creates an list of all of the dates to create the points, so if a other month is used the dates are not hard coded

where the intorpliation happens 

In [ ]:
for i, date in enumerate(dates):
    # Define the output paths based on the current date
    out_feature_class = f"C:/Users/jurge/Desktop/Stellenbosch/2023(Hons)/Modules/GIT 716/Hackathon/Dataoutputs/WeatherDATAshapefile/dayfirst_{i}.dbf"#The path where the points will be saved and the name that will change for each itteration making them all unique and in this case will name them from 0 to 30
    output_feature_class = f"weather_points_{i}.shp"
    where_clause = f'"date" = \'{date}\''

    # Convert table to feature class using XYTableToPoint
    arcpy.management.XYTableToPoint(
        arcpy.analysis.TableSelect(input_features, out_feature_class, where_clause),
        output_feature_class, x_field, y_field
    )
    
    # Interpolate windspeed, percipitation and humidity using IDW
    outIDWwindspeed = Idw(output_feature_class, "wind_speed", cellsize, 2, RadiusVariable(12))
    outIDWwindspeed.save(f"{savelocation}InterpolateOutidwout_windspeed_{i}.tif")
    outIDWperc = Idw(output_feature_class, "rain_sum", cellsize, 2, RadiusVariable(12))
    outIDWperc.save(f"{savelocation}InterpolateOutidwout_percipitation_{i}.tif")
    outIDWhumd = Idw(output_feature_class, "humidity_m", cellsize, 2, RadiusVariable(12))
    outIDWhumd.save(f"{savelocation}InterpolateOutidwout_humidity_{i}.tif")

Where composites are made for wind

In [ ]:
output_string_wind = ""

for i in range(len(dates)):
    interpolated_filename = f"C:/Users/jurge/Desktop/Stellenbosch/2023(Hons)/Modules/GIT 716/Hackathon/Dataoutputs/InterpolateOut/InterpolateOutidwout_windspeed_{i}.tif"
    output_string_wind += interpolated_filename
    if i < len(dates) - 1:
        output_string_wind += "; "
        
arcpy.CompositeBands_management(output_string_wind ,"windspeedComposite.tif")

Where composites are made for humidity

In [ ]:
output_string_humd = ""

for i in range(len(dates)):
    interpolated_filename = f"C:/Users/jurge/Desktop/Stellenbosch/2023(Hons)/Modules/GIT 716/Hackathon/Dataoutputs/InterpolateOut/InterpolateOutidwout_humidity_{i}.tif"
    output_string_humd += interpolated_filename
    if i < len(dates) - 1:
        output_string_humd += "; "
        
arcpy.CompositeBands_management(output_string_humd ,"humidityComposite.tif")


Where composites are made for percipitation

In [ ]:
output_string_perc = ""

for i in range(len(dates)):
    interpolated_filename = f"C:/Users/jurge/Desktop/Stellenbosch/2023(Hons)/Modules/GIT 716/Hackathon/Dataoutputs/InterpolateOut/InterpolateOutidwout_percipitation_{i}.tif"
    output_string_perc += interpolated_filename
    if i < len(dates) - 1:
        output_string_perc += "; "
        
arcpy.CompositeBands_management(output_string_perc ,"percComposite.tif")